# 1. Load Library

In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0

import keras
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation

# this is pandas table width setting
pd.set_option('display.max_columns', None) 

# 2. Load Data

In [115]:
# load train1, test1, rul1 pickle to have a test

train = pd.read_pickle('data/train.pkl')
test = pd.read_pickle('data/test.pkl')

# prepare feature list to each model

feature_s = ['cycle_norm','setting1','setting2','s2','s3','s4','s7','s8','s9','s11','s12','s13','s14','s15','s17','s20','s21']
feature = ['s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']
feature_ts = ['cycle_norm','setting1','setting2','setting3','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21']

### Training Data for Time Series

In [116]:
def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

sequence_length = 50
seq_gen = (list(gen_sequence(train[train['id']==id], sequence_length, feature_s)) for id in train['id'].unique())
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)
print('seq_array shape: ',seq_array.shape)

seq_array shape:  (15631, 50, 17)


# 3. Regression LSTM

### Training Label for Time Series

In [117]:
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

label_gen = [gen_labels(train[train['id']==id], sequence_length, ['rul']) for id in train['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
print('label_array shape: ', label_array.shape)

label_array shape:  (15631, 1)


### Model

In [142]:
# define path to save model
model_path = 'regression_model.h5'           
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# input/output size
nb_features = seq_array.shape[2]
nb_out = label_array.shape[1]

# modeling
model = Sequential()
model.add(LSTM(input_shape=(sequence_length, nb_features), units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=nb_out))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error', optimizer='rmsprop',metrics=['mae',r2_keras])
print(model.summary())

# fitting
history = model.fit(seq_array, label_array, epochs=100, batch_size=250, validation_split=0.05, verbose=2,
          callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min'),
                       keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, mode='min', verbose=0)])
print(history.history.keys())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_39 (LSTM)               (None, 50, 100)           47200     
_________________________________________________________________
dropout_39 (Dropout)         (None, 50, 100)           0         
_________________________________________________________________
lstm_40 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dropout_40 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 51        
_________________________________________________________________
activation_20 (Activation)   (None, 1)                 0         
Total params: 77,451
Trainable params: 77,451
Non-trainable params: 0
_________________________________________________________________
None
T

Epoch 42/100
 - 23s - loss: 990.0174 - mean_absolute_error: 19.5593 - r2_keras: 0.6977 - val_loss: 867.0238 - val_mean_absolute_error: 18.7088 - val_r2_keras: 0.7060
Epoch 43/100
 - 24s - loss: 969.8775 - mean_absolute_error: 19.4142 - r2_keras: 0.7039 - val_loss: 924.0018 - val_mean_absolute_error: 18.8953 - val_r2_keras: 0.7028
Epoch 44/100
 - 23s - loss: 934.6582 - mean_absolute_error: 19.0223 - r2_keras: 0.7142 - val_loss: 855.9780 - val_mean_absolute_error: 19.6254 - val_r2_keras: 0.6710
Epoch 45/100
 - 24s - loss: 909.9654 - mean_absolute_error: 18.8153 - r2_keras: 0.7218 - val_loss: 819.7569 - val_mean_absolute_error: 19.0516 - val_r2_keras: 0.6724
Epoch 46/100
 - 24s - loss: 867.1851 - mean_absolute_error: 18.4087 - r2_keras: 0.7352 - val_loss: 1053.7858 - val_mean_absolute_error: 18.5979 - val_r2_keras: 0.7085
Epoch 47/100
 - 24s - loss: 837.1513 - mean_absolute_error: 18.0089 - r2_keras: 0.7444 - val_loss: 830.4473 - val_mean_absolute_error: 18.7659 - val_r2_keras: 0.6397
Epo

### Check Overfitting

In [143]:
def history_plot(hist1, hist2, title, ylabl, xlabl):
    fig_acc = plt.figure(figsize=(12,4))
    plt.plot(history.history[hist1])
    plt.plot(history.history[hist2])
    plt.title(title)
    plt.ylabel(ylabl)
    plt.xlabel(xlabl)
    plt.legend(['train', 'test'], loc='upper left')
    return plt

### Performance

In [146]:
scores = model.evaluate(seq_array, label_array, batch_size=250, verbose=2)
print('\nMAE: {}'.format(scores[1]))
print('\nR^2: {}'.format(scores[2]))

y_pred = model.predict(seq_array,verbose=2, batch_size=250)
y_true = label_array
test_set = pd.DataFrame(y_pred)


MAE: 13.877024712256587

R^2: 0.8054496281344223


### Validation

# 4. Classification LSTM

### Training Label for Time Series

In [179]:
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

label_gen = [gen_labels(train[train['id']==id], sequence_length, ['w1']) for id in train['id'].unique()]
label_array = np.concatenate(label_gen).astype(np.float32)
print('label_array shape: ', label_array.shape)

label_array shape:  (15631, 1)


### Model